In [1]:
import pandas as pd

In [2]:
data_url = pd.read_csv('UrlDataSet.csv',encoding='latin1')

In [3]:
data_url.shape

(11055, 31)

In [4]:
data_url

,having_IP_Address,URL_Length,Shortining_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registeration_length,Favicon,...,popUpWidnow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,-1,1,1,1,-1,-1,-1,-1,-1,1,...,1,1,-1,-1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,-1,0,1,-1,1,...,1,1,-1,-1,0,-1,1,1,1,-1
2,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,-1,1,-1,1,0,-1,-1
3,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
4,1,0,-1,1,1,-1,1,1,-1,1,...,-1,1,-1,-1,0,-1,1,1,1,1
5,-1,0,-1,1,-1,-1,1,1,-1,1,...,1,1,1,1,1,-1,1,-1,-1,1
6,1,0,-1,1,1,-1,-1,-1,1,1,...,1,1,1,-1,-1,-1,1,0,-1,-1
7,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,-1,-1,0,-1,1,0,1,-1
8,1,0,-1,1,1,-1,1,1,-1,1,...,1,1,1,-1,1,1,1,0,1,1
9,1,1,-1,1,1,-1,-1,1,-1,1,...,1,1,1,-1,0,-1,1,0,1,-1


In [5]:
n_comp = 15

In [6]:
from sklearn.decomposition import PCA, FastICA, TruncatedSVD
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(data_url.drop(["Result"], axis=1))

In [7]:
from sklearn.decomposition import PCA, FastICA, TruncatedSVD
ica = FastICA(n_components=n_comp, random_state=100)
ica2_results_train = ica.fit_transform(data_url.drop(["Result"], axis=1))

In [8]:
from sklearn.decomposition import TruncatedSVD
tsvd = TruncatedSVD(n_components=n_comp, random_state=420)
tsvd_results_train = tsvd.fit_transform(data_url.drop(["Result"], axis=1))

In [9]:
from sklearn.random_projection import SparseRandomProjection
srp = SparseRandomProjection(n_components=n_comp, dense_output=True, random_state=420)
srp_results_train = srp.fit_transform(data_url.drop(["Result"], axis=1))

In [10]:
from sklearn.random_projection import GaussianRandomProjection
grp = GaussianRandomProjection(n_components=n_comp, eps=0.1, random_state=420)
grp_results_train = grp.fit_transform(data_url.drop(["Result"], axis=1))

In [11]:
data_url_f = pd.DataFrame(data_url["Result"])

In [12]:
for i in range(1, n_comp + 1):
    data_url_f['srp_' + str(i)] = pca2_results_train[:, i - 1]
    data_url_f['srp_' + str(i)] = ica2_results_train[:, i - 1]
    data_url_f['srp_' + str(i)] = tsvd_results_train[:, i - 1]
    data_url_f['tsvd_' + str(i)] = srp_results_train[:, i - 1]
    data_url_f['grp_' + str(i)] = grp_results_train[:, i - 1]

In [13]:
target = ["Result"]

In [16]:
features = [x for x in data_url_f.columns if x not in target ]

In [18]:
X = data_url_f[features].values
y = data_url_f[target].values.ravel()

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=5678)

In [20]:
import lightgbm as lgb

train_set = lgb.Dataset(X_train, y_train)
test_sets = lgb.Dataset(X_test, y_test, reference=train_set)

In [22]:
params = {
    'task': 'train',
    'boosting_type': 'rf',
    'objective': 'binary',
    'metric': { 'mae' },
    'num_leaves': 250,
    #'min_data_in_leaf': 30,
    'learning_rate': 0.01,
    'max_depth': 6,
    'feature_fraction': 0.8,
    'bagging_freq': 1,
    'bagging_fraction': 0.8,
    'use_missing': False,
    #'is_unbalance': True,
    'verbose': 1
    }

import time
start_time = time.clock()
    
model = lgb.train(
    params,
    train_set,
    num_boost_round=15000,
    valid_sets=test_sets,
    early_stopping_rounds=50,
    #categorical_feature=index_categorical,
    verbose_eval=1)

[1]	valid_0's l1: 0.657339
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's l1: 0.653319
[3]	valid_0's l1: 0.657829
[4]	valid_0's l1: 0.656869
[5]	valid_0's l1: 0.656633
[6]	valid_0's l1: 0.658472
[7]	valid_0's l1: 0.658128
[8]	valid_0's l1: 0.657978
[9]	valid_0's l1: 0.658572
[10]	valid_0's l1: 0.661158
[11]	valid_0's l1: 0.660805
[12]	valid_0's l1: 0.661562
[13]	valid_0's l1: 0.662155
[14]	valid_0's l1: 0.661836
[15]	valid_0's l1: 0.662139
[16]	valid_0's l1: 0.6637
[17]	valid_0's l1: 0.663581
[18]	valid_0's l1: 0.663362
[19]	valid_0's l1: 0.662969
[20]	valid_0's l1: 0.662439
[21]	valid_0's l1: 0.662338
[22]	valid_0's l1: 0.662852
[23]	valid_0's l1: 0.662276
[24]	valid_0's l1: 0.662032
[25]	valid_0's l1: 0.66185
[26]	valid_0's l1: 0.661662
[27]	valid_0's l1: 0.661508
[28]	valid_0's l1: 0.661208
[29]	valid_0's l1: 0.661314
[30]	valid_0's l1: 0.66132
[31]	valid_0's l1: 0.661595
[32]	valid_0's l1: 0.661701
[33]	valid_0's l1: 0.66161
[34]	valid_0's l1: 0.661333


In [26]:
estado_m = model.predict(data_url[features],num_iteration=model.best_iteration)

In [27]:
estado_m

array([0.11920292, 0.21981243, 0.15941916, ..., 0.28342661, 0.11920292,
       0.11920292])